In [2]:
import torch
import os
import numpy as np
import torchaudio
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

c:\Users\amos.zuercher\.conda\envs\ai\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
DATA_PATH = './training_data/'
WAV_PATH = os.path.join(DATA_PATH, 'genres_original')
PNG_PATH = os.path.join(DATA_PATH, 'images_original')

In [4]:
### Arguments
### path: path of either wav or png directory
### Returns
### dictionary of 'genre' -> number
def extract_classes(path=WAV_PATH):
    classes = {}
    i = 0
    for genre in os.listdir(path):
        classes[genre] = i
        i += 1
    return classes
CLASSES = extract_classes()
print(CLASSES)

{'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}


In [10]:
'''
Returns a dataset with audio data and its classification

{
    'train': [[f0-fn],[f0-fn],...],
    'target': [1,3,...]
}
'''
def getTrainDataSet(path, duration=10, sr=22050):
    genres = os.listdir(path)
    n_frame = duration * sr
    set = {'data': [], 'target': []}    
    for genre in genres:
        genre_path = os.path.join(path, genre)
        files = os.listdir(genre_path)
        for file in files:
            file_path = os.path.join(genre_path, file)
            audio_frames = torchaudio.load(file_path, num_frames=n_frame)            
            set['data'].append(audio_frames[0])
            set['target'].append(CLASSES[genre])
    return set



ds = getTrainDataSet(WAV_PATH)
print(ds)

{'data': [tensor([[0.0073, 0.0166, 0.0076,  ..., 0.0762, 0.0768, 0.0813]]), tensor([[-0.0201, -0.0175,  0.0042,  ..., -0.0197, -0.0295, -0.0363]]), tensor([[-0.1342, -0.1949, -0.0426,  ..., -0.0705, -0.0361, -0.0229]]), tensor([[-0.2320, -0.2317, -0.1979,  ...,  0.0184,  0.0374, -0.0263]]), tensor([[-0.1692, -0.0217,  0.0196,  ..., -0.2574, -0.3943, -0.3089]]), tensor([[-0.0092, -0.0118, -0.0137,  ..., -0.0362, -0.0292, -0.0204]]), tensor([[-0.1224, -0.0492,  0.0444,  ..., -0.1051, -0.1873, -0.0345]]), tensor([[-0.0887, -0.0952, -0.1028,  ..., -0.1279, -0.0767, -0.0520]]), tensor([[0.0104, 0.0078, 0.0389,  ..., 0.0104, 0.0135, 0.0150]]), tensor([[-0.0334, -0.0549, -0.0544,  ..., -0.3696, -0.3372, -0.2458]])], 'target': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}


In [12]:
print(ds['data'][0].shape)

torch.Size([1, 220500])


In [13]:
BATCH_SIZE = 64

In [14]:
train_target = ds['data'] # train ist ein DF
train = torch.tensor(ds['target']values.astype(np.float32))
train_tensor = TensorDataset(train, train_target)
test_tensor = TensorDataset(train, train_target)

train_loader = DataLoader(dataset = train_tensor, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(dataset = train_tensor, batch_size = BATCH_SIZE, shuffle = True)

AttributeError: 'list' object has no attribute 'size'

In [ ]:
for X, y in train_loader:
    print(X.shape)
    break

In [ ]:
torch.full(size=(9,), fill_value=1)

In [ ]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.Softmax()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()